In [1]:
#facebook user access token
USER_ACCESS_TOKEN = 'EAAEDdNZCNQMABALCHiqYlpYEGZCy02jny4QSEUPafJsWSWoeqmQ5RZC98YdoYcDjqziW5htNquTfMNRYZChFysLtHKxAoyLLKXyE5wDQ5jiNOMtjPcWnxD5kbqsX2YyhZAIPNkWTbxQ4ykZAM9ZCtSt0EESdv8pRizeQ6NZCGHtbWLZACx2CxTSEmEvdu9z1nodQZD'

In [4]:
#facebook group ID

GROUP_ID = '2709640489282846'

In [3]:
pip install urllib3

Note: you may need to restart the kernel to use updated packages.


In [7]:
parameters = {'access_token': USER_ACCESS_TOKEN}

group_url = 'https://graph.facebook.com/%d/' % GROUP_ID

r = requests.get(group_url, params=parameters)

TypeError: %d format: a number is required, not str

In [8]:
app_id = '285275482570944'
app_secret = '082f1b26a80803f4095be755e339e859'
url = "https://graph.facebook.com/oauth/access_token?grant_type=" \
    "client_credentials&client_id=%s&client_secret=%s" % \
    (client_id, client_secret)
r = requests.get(url)

access_token = r.text.split('=')[1]

NameError: name 'client_id' is not defined

In [9]:
pip install django-allauth

  Using cached https://files.pythonhosted.org/packages/01/a5/fb3dad18422fcd4241d18460a1fe17542bfdeadcf74e3861d1a2dfc9e459/Django-3.1.1-py3-none-any.whl
  Stored in directory: C:\Users\Administrator\AppData\Local\pip\Cache\wheels\46\3e\b4\592ed6417e7bba12cb05f68f8387384932b87327719e80a40d
Successfully built django-allauth
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install requirements.txt

Note: you may need to restart the kernel to use updated packages.


  Could not find a version that satisfies the requirement requirements.txt (from versions: )
No matching distribution found for requirements.txt


In [3]:
from openpyxl import Workbook

In [4]:
import json
import facebook
from openpyxl import load_workbook
from urllib.parse import urlencode
from urllib.request import urlopen
import os.path

In [29]:
pwd

'C:\\Users\\Administrator\\Desktop\\Company Hire\\InfoAxon'

In [34]:
filename_root ='C:/Users/Administrator/Desktop/Company Hire/InfoAxon'

In [35]:
 #Function that gets all sustainability keywords from an excel file and returns in a list
def getSustainabilityKeywords(filename):

    # open the keyword excel workbook
    kw_wb_input = load_workbook(filename=filename)

    # select SusMon_Keywords.xlsx
    kw_sheet = kw_wb_input.active

    # Get all the keywords from the first column
    m_row = kw_sheet.max_row

    # Initialize keyword list
    keywordList = []

    for i in range(1, m_row + 1):
        cell_obj = kw_sheet.cell(row=i, column=1)
        keywordList.append(cell_obj.value)

    return keywordList

In [36]:
def getInputData(filename):

    # open the excel input workbook
    wb_input = load_workbook(filename=filename, read_only=True)

    # select FBPage_Input.xlsx
    sheet = wb_input.active

    # get Page ID
    pageIdLabel = sheet['B1']
    pageIdValue = sheet['B2']
    pageIdValue = round(pageIdValue.value)

    # get Brand name
    pageNameLabel = sheet['C1']
    pageNameValue = sheet['C2']
    pageNameValue = pageNameValue.value

    # get Access Token
    pageAccessTokenLabel = sheet['D1']
    pageAccessTokenValue = sheet['D2']
    pageAccessTokenValue = pageAccessTokenValue.value

    # Store ID and Access token in a dictionary
    d = dict()
    d['brands'] = [{'id':pageIdValue}] #at the moment we are only expecting a single brand, but the rest of the code expects multiuple brands
    d['accessToken'] = pageAccessTokenValue
    return d

In [37]:
# Function that calls Input Data and Keywords functions 
def loadConfig():
    input_filename = filename_root + '/FBPage_Input.xlsx'
    keyword_filename = filename_root + '/SusMon_Keywords.xlsx'
    config = getInputData(input_filename)
    config['keywords'] = getSustainabilityKeywords(keyword_filename)
    return config

In [38]:
# Function that calls Facebook's Graph API to grab all the posts for a particular brand page
def getPostsForBrand(brandID, token):
    uri="https://graph.facebook.com/v4.0/{}/posts/".format(str(brandID))
    fields = "id,created_time,message,shares.summary(true).limit(0),comments,likes.summary(true),reactions.type(LOVE).limit(0).summary(total_count).as(Love),reactions.type(WOW).limit(0).summary(total_count).as(Wow),reactions.type(HAHA).limit(0).summary(total_count).as(Haha),reactions.type(SAD).limit(0).summary(1).as(Sad),reactions.type(ANGRY).limit(0).summary(1).as(Angry)"
    url= "{uri}?fields={fields}&access_token={token}&limit={limit}"\
    .format(**{'uri':uri,'fields':fields,'token': str(token), 'limit':50 })
    
  #url = "https://graph.facebook.com/v4.0/{pageID}/posts/?fields=id,created_time,message,shares.summary(true).limit(0),comments.summary(true).limit(0),likes.summary(true),reactions.type(LOVE).limit(0).summary(total_count).as(Love),reactions.type(WOW).limit(0).summary(total_count).as(Wow),reactions.type(HAHA).limit(0).summary(total_count).as(Haha),reactions.type(SAD).limit(0).summary(1).as(Sad),reactions.type(ANGRY).limit(0).summary(1).as(Angry)&access_token={token}&limit=50".format(**{'pageID': str(pageId), 'token': str(token)})
    try:
        facebook_connection = urlopen(url)
        data = facebook_connection.read().decode('utf8')
        json_object = json.loads(data)
        
    except Exception as ex:
        print(ex)
    return json_object["data"]

In [39]:
# Exports post data to a csv
def exportResultsToSCV(results):
    
    wb = Workbook()

  # List for all post wtih comments
    postWithComments = []

  # set file path
    filePath = filename_root+'/FBOutput.xlsx'

  # Save file in the path
    wb.save(filePath)

  # Load workbook
    wb = load_workbook(filePath)

    sheet = wb.active

  # Create Headers in created output file in a row
    sheet['A1'] = 'Brand ID'
    sheet['B1'] = 'Post ID'
    sheet['C1'] = 'Created Time'
    sheet['D1'] = '#Comments'
    sheet['E1'] = '#like'
    sheet['F1'] = '#love'
    sheet['G1'] = '#wow'
    sheet['H1'] = '#haha'
    sheet['I1'] = '#sad'
    sheet['J1'] = '#angry'
    sheet['K1'] = 'message'
    sheet['L1'] = 'Shares'

  # Save data retreived from Facebook in next row
    row = 2

    for brand in results:
        brandID = brand['brandId']

        for post in brand['posts']:
            sheet['A'+str(row)] = brandID
            sheet['B'+str(row)] = post['id']
            sheet['C'+str(row)] = post['created_time']
        if 'comments' in post:
            sheet['D'+str(row)] = len(post['comments']['data'])
        else:
            sheet['D'+str(row)] = '0'
            sheet['E'+str(row)] = len(post['likes']['data'])
            sheet['F'+str(row)] = post['Love']['summary']['total_count']
            sheet['G'+str(row)] = post['Wow']['summary']['total_count']
            sheet['H'+str(row)] = post['Haha']['summary']['total_count']
            sheet['I'+str(row)] = post['Sad']['summary']['total_count']
            sheet['J'+str(row)] = post['Angry']['summary']['total_count']
            sheet['K'+str(row)] = post['message']
            if 'shares' in post:
                sheet['L'+str(row)] = post['shares']['count']
            else:
                sheet['L'+str(row)] = '0'
            if 'comments' in post:
                for comment in post['comments']['data']:
                    postWithComments.append(comment)          
            row = row + 1 

    storePostComments(postWithComments)

    wb.save(filePath)


In [43]:
# Function that store Post comments in an excel file
def storePostComments(commentDetails):
    wb = Workbook()
    filePath = filename_root+'/PostComments.xlsx'
    wb.save(filePath)
    wb = load_workbook(filePath)
    sheet = wb.active
    sheet['A1'] = 'Post ID'
    sheet['B1'] = 'Comment Datetime'
    sheet['C1'] = 'Comment Content'
    row = 2
    for commentDetail in commentDetails:
        sheet['A'+str(row)] = commentDetail['id']
        sheet['B'+str(row)] = commentDetail['created_time']
        sheet['C'+str(row)] = commentDetail['message']
        row += 1

    wb.save(filePath)

In [44]:
def main():
    config = loadConfig()
    results = []
    for brand in config['brands']:
        relevantPosts = []
        posts = getPostsForBrand(brand['id'], config['accessToken'])
        for post in posts:
            if isRelevant(post, config['keywords']):
                relevantPosts.append(post)

        results.append({'brandId':brand['id'], 'posts':relevantPosts})

    exportResultsToSCV(results)

if __name__ == '__main__':
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Administrator/Desktop/Company Hire/InfoAxon/FBPage_Input.xlsx'